In [1]:
import QuantLib as ql

In [3]:
calendar = ql.UnitedKingdom()
bdc = ql.ModifiedFollowing
evaluationDate = ql.Date(13, ql.August, 2007)
evaluationDate = calendar.adjust(evaluationDate)
ql.Settings.instance().evaluationDate = evaluationDate

In [9]:
fromDate = ql.Date(1, ql.January, 2005)
toDate = ql.Date(13, ql.August, 2007)
rpiSchedule = ql.MakeSchedule(effectiveDate=fromDate,terminationDate=toDate,tenor=ql.Period("1M"),calendar=ql.UnitedKingdom(),convention=ql.ModifiedFollowing)



In [13]:
fixData = [ 189.9, 189.9, 189.6, 190.5, 191.6, 192.0,
    192.2, 192.2, 192.6, 193.1, 193.3, 193.6,
    194.1, 193.4, 194.2, 195.0, 196.5, 197.7,
    198.5, 198.5, 199.2, 200.1, 200.4, 201.1,
    202.7, 201.6, 203.1, 204.4, 205.4, 206.2,
    207.3]

In [15]:
hz = ql.RelinkableZeroInflationTermStructureHandle()

In [16]:
interp = False

In [17]:
iiUKRPI = ql.UKRPI(interp, hz)

In [18]:
for i in range(0, len(fixData)):
    iiUKRPI.addFixing(rpiSchedule[i], fixData[i])

In [20]:
nominalTS = ql.FlatForward(ql.Date(13, ql.August, 2007), 0.05, ql.Actual360())

In [23]:
zcData = [
    [ ql.Date(13, ql.August, 2008), 2.93 ],
    [ ql.Date(13, ql.August, 2009), 2.95 ],
    [ ql.Date(13, ql.August, 2010), 2.965 ],
    [ ql.Date(15, ql.August, 2011), 2.98 ],
    [ ql.Date(13, ql.August, 2012), 3.0 ],
    [ ql.Date(13, ql.August, 2014), 3.06 ],
    [ ql.Date(13, ql.August, 2017), 3.175 ],
    [ ql.Date(13, ql.August, 2019), 3.243 ],
    [ ql.Date(15, ql.August, 2022), 3.293 ],
    [ ql.Date(14, ql.August, 2027), 3.338 ],
    [ ql.Date(13, ql.August, 2032), 3.348 ],
    [ ql.Date(15, ql.August, 2037), 3.348 ],
    [ ql.Date(13, ql.August, 2047), 3.308 ],
    [ ql.Date(13, ql.August, 2057), 3.228 ]
    ]

In [25]:
observationLag = ql.Period(2,ql.Months)
dc = ql.Thirty360()
frequency = ql.Monthly

In [30]:
instruments = []
for i in range(0, len(zcData)):
    maturity = zcData[i][0]
    quote = ql.QuoteHandle(ql.SimpleQuote(zcData[i][1]/100.0))
    anInstrument = ql.ZeroCouponInflationSwapHelper(quote, observationLag, maturity,
                calendar, bdc, dc, iiUKRPI, ql.YieldTermStructureHandle(nominalTS))
    instruments += [anInstrument]

In [33]:
baseZeroRate = zcData[0][1]/100.0

pZITS = ql.PiecewiseZeroInflation(evaluationDate, calendar, dc, observationLag,
                    frequency, iiUKRPI.interpolated(), baseZeroRate,
                    ql.YieldTermStructureHandle(nominalTS), instruments)

In [35]:
# first check that the zero rates on the curve match the data
# and that the helpers give the correct impled rates
eps = 0.00000001
forceLinearInterpolation = False

for i in range(0, len(zcData)):
    if (abs(zcData[i][1]/100.0 - pZITS.zeroRate(zcData[i][0], observationLag, forceLinearInterpolation)) > eps):
        print ("ERROR rate ",i)
    if (abs(instruments[i].impliedQuote() - zcData[i][1]/100.0) > eps):
        print ("ERROR quote ",i)

In [36]:
hz.linkTo(pZITS)

In [38]:
fromDate = hz.baseDate()
toDate = hz.maxDate()-1*ql.Months # a bit of margin for adjustments

In [40]:
toDate

Date(28,6,2057)

In [41]:
testIndex = ql.MakeSchedule(effectiveDate=fromDate,
                              terminationDate=toDate,
                              tenor=ql.Period("1M"),
                              calendar=ql.UnitedKingdom(),
                              convention=ql.ModifiedFollowing)

In [49]:
bd = hz.baseDate()
bf = iiUKRPI.fixing(bd)

for i in range(0, len(testIndex)):
    d = testIndex[i]
    z = hz.zeroRate(d, ql.Period(0,ql.Days))
    t = hz.dayCounter().yearFraction(bd, d)
    #if not (iiUKRPI.interpolated()): # because fixing constant over period
    #    t = hz.dayCounter().yearFraction(bd,
    #        inflationPeriod(d, iiUKRPI.frequency()).first)
    

NameError: name 'inflationPeriod' is not defined

In [53]:
test = ql.ZeroInflationCurve(evaluationDate, ql.UnitedKingdom(), ql.Actual365Fixed(),
                            ql.Period("1Y"), ql.Annual, True, ql.YieldTermStructureHandle(nominalTS),
                            [ql.Date(1,1,2006), ql.Date(1,1,2021)], [0.02, 0.02])

In [55]:
test.zeroRate(1.0, True)

0.02

In [56]:
test = ql.YoYInflationCurve(evaluationDate, ql.UnitedKingdom(), ql.Actual365Fixed(),
                            ql.Period("1Y"), ql.Annual, True, ql.YieldTermStructureHandle(nominalTS),
                            [ql.Date(1,1,2006), ql.Date(1,1,2021)], [0.02, 0.02])

In [57]:
test.yoyRate(1.0, True)

0.02